In [1]:
# Import all the required libraries

import tweepy
import csv
import pandas as pd
import os
import wget
from textblob import TextBlob
from sqlite3 import connect

import warnings
warnings.filterwarnings("ignore")

In [2]:
bearer_token="AAAAAAAAAAAAAAAAAAAAAOKciwEAAAAA0gkjQ%2FzQuU7YnhNv%2BPm%2BV%2Brkzxo%3D30o9PlKjyGXxKfZSuf3SyfrC222cQA7ju5BBIzHFtWGnCSR8PH"

In [3]:
# authentication to Twitter API

auth = tweepy.OAuth2BearerHandler(bearer_token)
api = tweepy.API(auth)
client = tweepy.Client(bearer_token)

In [4]:
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_key, access_secret)

In [4]:
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)


In [12]:
# definitions to extract data and load to a dataframe

def get_twitter_data_for_query(query,count):
    
    outtweets = []
    outusers = []
    outtags = []
    
    "funtion returns tweets dataframe given the query and count"
    
    for tweet in tweepy.Cursor(api.search_tweets,q=query,count=count, 
                           lang="en").items():

        outtweets.append([tweet.id_str,tweet.user.screen_name,tweet.text,tweet.created_at,tweet.retweet_count,tweet.favorite_count])
        outusers.append([tweet.user.id_str,tweet.user.screen_name,tweet.user.name,tweet.user.profile_image_url,tweet.user.description,tweet.user.followers_count,tweet.user.created_at])
        outtags.append([tweet.id_str,','.join([tag_dict['text'] for tag_dict in tweet.entities.get('hashtags',None)])])
        
    tweets_df_for_curr_query=pd.DataFrame(outtweets,columns = ["tweet_id","twitter_handle","tweet_text","created_at","retweet_count","likes_count"])
    users_df_for_curr_query = pd.DataFrame(outusers,columns = ["user_id","user_handle","user_name","user_profile_image","description","followers_count","created_at"])
    tags_df_for_curr_query = pd.DataFrame(outtags,columns=["tweet_id","tags"])

    return tweets_df_for_curr_query,users_df_for_curr_query,tags_df_for_curr_query


def get_sentiment_data(disease,medicine,count,query):
    
    "function creates tweets data"
    "to extracted tweets data function appends sentiment score"
    query_sentiment_curr_df = pd.DataFrame()
    
    medicine_tweet_data_for_curr_query,users_df_for_curr_query,tags_df_for_curr_query = get_twitter_data_for_query(query,count)
    
    query_sentiment_curr_df['tweet_id'] = medicine_tweet_data_for_curr_query['tweet_id']
    query_sentiment_curr_df['query'] = query
    query_sentiment_curr_df['sentiment'] = medicine_tweet_data_for_curr_query['tweet_text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
    
    return medicine_tweet_data_for_curr_query,users_df_for_curr_query,tags_df_for_curr_query,query_sentiment_curr_df


def get_all_medicine_sentiment_data(search_dict,count):
    
    all_medicine_tweet_data= pd.DataFrame()
    all_users_df= pd.DataFrame()
    all_tags_df= pd.DataFrame()
    all_query_sentiment_df = pd.DataFrame()
    
    for disease in search_dict.keys():
        for medicine in search_dict[disease]:
            query = medicine
            print("searched query ",query)
            medicine_tweet_data_for_curr_query,users_df_for_curr_query,tags_df_for_curr_query,query_sentiment_curr_df = get_sentiment_data(disease,medicine,count,query)
            
            all_medicine_tweet_data = pd.concat([all_medicine_tweet_data,medicine_tweet_data_for_curr_query])
            all_users_df = pd.concat([all_users_df,users_df_for_curr_query])
            all_tags_df = pd.concat([all_tags_df,tags_df_for_curr_query])
            all_query_sentiment_df = pd.concat([all_query_sentiment_df,query_sentiment_curr_df])
        

    return all_medicine_tweet_data,all_users_df,all_tags_df,all_query_sentiment_df

def get_all_hospitals_sentiment_data(search_dict,count):
    
    all_hospitals_tweet_data= pd.DataFrame()
    all_users_df= pd.DataFrame()
    all_tags_df= pd.DataFrame()
    all_query_sentiment_df = pd.DataFrame()
    
    all_setiment_data= pd.DataFrame()
    for disease in search_dict.keys():
        for hospital in search_dict[disease]:
            query = hospital + " AND " + disease
            print("searched query ",query)
            
            hospitals_tweet_data_for_curr_query,users_df_for_curr_query,tags_df_for_curr_query,query_sentiment_curr_df = get_sentiment_data(disease,hospital,count,query)
            
            all_hospitals_tweet_data = pd.concat([all_hospitals_tweet_data,hospitals_tweet_data_for_curr_query])
            all_users_df = pd.concat([all_users_df,users_df_for_curr_query])
            all_tags_df = pd.concat([all_tags_df,tags_df_for_curr_query])
            all_query_sentiment_df = pd.concat([all_query_sentiment_df,query_sentiment_curr_df])
            
    return all_hospitals_tweet_data,all_users_df,all_tags_df,all_query_sentiment_df

def get_mean_sentiment_scores_from_all_sentiment_data(data,col):
    return pd.DataFrame(data.groupby(['disease',col])['sentiment'].mean()).reset_index(drop = False).sort_values(by = ['disease','sentiment'],ascending = False)

def search_dict2search_df(search_dict):
    return pd.DataFrame([[k,v1] for k,v in search_dict.items() for v1 in v],columns = ['disease','medicine'])


### Disease - Medicine

In [13]:
# Connecting to a database to store the extracted data

conn = connect('Medicines Database DMDD2.db') 
diseases_df = pd.read_sql('SELECT * FROM Diseases',conn)
drugs_df = pd.read_sql('SELECT * FROM Drugs', conn)
diseases_df

,DiseaseId,DiseaseName,Deaths
0,1,Cancer,12934
1,2,Heart Disease,12140
2,3,Chronic Lower Respiratory Disease,2842
3,4,Stroke,2367
4,5,Alzheimer’s disease,1041
5,6,Flu/Pneumonia,1433
6,7,Diabetes,1321
7,8,Kidney Disease,1193
8,9,Septicemia,864


In [14]:
drugs_df

,DrugId,DrugName,DiseaseId
0,1,Perjeta,1
1,2,Xtandi,1
2,3,Avastin,1
3,4,Herceptin,1
4,5,Gazyva,1
5,6,Jakafi,1
6,7,Venclexta,1
7,8,Rituxa,1
8,9,Enalapril,2
9,10,Fosinopril,2


In [15]:
merged_data=pd.merge(diseases_df,drugs_df,on='DiseaseId')
merged_data

,DiseaseId,DiseaseName,Deaths,DrugId,DrugName
0,1,Cancer,12934,1,Perjeta
1,1,Cancer,12934,2,Xtandi
2,1,Cancer,12934,3,Avastin
3,1,Cancer,12934,4,Herceptin
4,1,Cancer,12934,5,Gazyva
5,1,Cancer,12934,6,Jakafi
6,1,Cancer,12934,7,Venclexta
7,1,Cancer,12934,8,Rituxa
8,2,Heart Disease,12140,9,Enalapril
9,2,Heart Disease,12140,10,Fosinopril


In [16]:
medicines_search_dict = {}
group_data=merged_data.groupby(['DiseaseName'])['DrugName'].apply(list)
for i,j in group_data.iteritems():
    medicines_search_dict[i]=j
medicines_search_dict

{'Cancer': ['Perjeta',
  'Xtandi',
  'Avastin',
  'Herceptin',
  'Gazyva',
  'Jakafi',
  'Venclexta',
  'Rituxa'],
 'Chronic Lower Respiratory Disease': ['Plavix',
  'Clopidogrel',
  'Activase',
  'Bayer Aspirin',
  'Easprin',
  'Ecotrin',
  'Arthritis Pain',
  'Ascriptin',
  'Aspergum',
  'Genacote'],
 'Heart Disease': ['Enalapril',
  'Fosinopril',
  'Lisinopril',
  'Moexipril',
  'Perindopril',
  'Quinapril',
  'Ramipril',
  'Trandolapril',
  'Albuterol',
  'Beclomethasone',
  'Budesonide',
  'Fluticasone',
  'Ipratropium',
  'Tiotropium']}

In [17]:
all_medicine_tweet_data,all_medicine_users_df,all_medicine_tags_df,all_medicine_query_sentiment_df = get_all_medicine_sentiment_data(medicines_search_dict,1000)

searched query  Perjeta
searched query  Xtandi
searched query  Avastin
searched query  Herceptin
searched query  Gazyva
searched query  Jakafi
searched query  Venclexta
searched query  Rituxa
searched query  Plavix
searched query  Clopidogrel
searched query  Activase
searched query  Bayer Aspirin
searched query  Easprin
searched query  Ecotrin
searched query  Arthritis Pain
searched query  Ascriptin
searched query  Aspergum
searched query  Genacote
searched query  Enalapril
searched query  Fosinopril
searched query  Lisinopril
searched query  Moexipril
searched query  Perindopril
searched query  Quinapril
searched query  Ramipril
searched query  Trandolapril
searched query  Albuterol
searched query  Beclomethasone
searched query  Budesonide
searched query  Fluticasone
searched query  Ipratropium
searched query  Tiotropium


In [18]:
from IPython.display import display
for df in all_medicine_tweet_data,all_medicine_users_df,all_medicine_tags_df,all_medicine_query_sentiment_df:
    print(" ")
    print(len(df))
    display(df.head(2))

 
4632


,tweet_id,twitter_handle,tweet_text,created_at,retweet_count,likes_count
0,1590701484282241024,vimal_madani,Please reduce the price of perjeta as perjeta ...,2022-11-10 13:42:42+00:00,0,0
1,1589430238651764736,maperdoo,@WilsonMaryAnne1 @ThanksCancer I got congestiv...,2022-11-07 01:31:13+00:00,0,0


 
4632


,user_id,user_handle,user_name,user_profile_image,description,followers_count,created_at
0,2835840133,vimal_madani,vimal madani,http://abs.twimg.com/sticky/default_profile_im...,,0,2014-09-30 09:47:39+00:00
1,4330690359,maperdoo,Cmon Warnock,http://pbs.twimg.com/profile_images/1347168604...,Resistor. Bereaved Mom. Reluctant Pharmacist.\...,1165,2015-11-30 16:42:48+00:00


 
4632


,tweet_id,tags
0,1590701484282241024,
1,1589430238651764736,


 
4632


,tweet_id,query,sentiment
0,1590701484282241024,Perjeta,0.000000
1,1589430238651764736,Perjeta,-0.316667


### Hospital - Medcine

In [19]:
hospitals_df=pd.read_sql('SELECT * FROM Hospitals', conn)

hospitals_df['tmp'] = 1
diseases_df['tmp'] = 1
df = pd.merge(hospitals_df, diseases_df, on=['tmp'])
df = df.drop('tmp', axis=1)
df

# merged_hos_data=pd.merge(hospitals_df,diseases_df)
# merged_data

# hospitals_df = pd.read_sql('SELECT * FROM Hospitals', conn)
hospitals_search_dict = {}

hos_group_data=df.groupby(['DiseaseName'])['HospitalName'].apply(list)
for i,j in hos_group_data.iteritems():
    hospitals_search_dict[i]=j
hospitals_search_dict

{'Alzheimer’s disease': ['Massachusetts General Hospital',
  "Brigham and Women's Hospital",
  'Beth Israel Deaconess Medical Center',
  'Lahey Hospital and Medical Center',
  'Tufts Medical Center',
  "Women's Faulkner Hospital",
  'Mount Auburn Hospital',
  'Newton-Wellesley Hospital',
  'Baystate Medical Center',
  'UMass Memorial Medical Center',
  'South Shore Hospital-South Weymouth',
  'Cape Cod Hospital',
  'Adcare Hospital of Worcester',
  'Anna Jaques Hospital',
  'Arbour-Fuller Hospital',
  'Arbour Hospital'],
 'Cancer': ['Massachusetts General Hospital',
  "Brigham and Women's Hospital",
  'Beth Israel Deaconess Medical Center',
  'Lahey Hospital and Medical Center',
  'Tufts Medical Center',
  "Women's Faulkner Hospital",
  'Mount Auburn Hospital',
  'Newton-Wellesley Hospital',
  'Baystate Medical Center',
  'UMass Memorial Medical Center',
  'South Shore Hospital-South Weymouth',
  'Cape Cod Hospital',
  'Adcare Hospital of Worcester',
  'Anna Jaques Hospital',
  'Arbour

In [20]:
all_hospitals_tweet_data,all_hospitals_users_df,all_hospitals_tags_df,all_hospitals_query_sentiment_df = get_all_hospitals_sentiment_data(hospitals_search_dict,1000)


searched query  Massachusetts General Hospital AND Alzheimer’s disease
searched query  Brigham and Women's Hospital AND Alzheimer’s disease
searched query  Beth Israel Deaconess Medical Center AND Alzheimer’s disease
searched query  Lahey Hospital and Medical Center AND Alzheimer’s disease
searched query  Tufts Medical Center AND Alzheimer’s disease
searched query  Women's Faulkner Hospital AND Alzheimer’s disease
searched query  Mount Auburn Hospital AND Alzheimer’s disease
searched query  Newton-Wellesley Hospital AND Alzheimer’s disease
searched query  Baystate Medical Center AND Alzheimer’s disease
searched query  UMass Memorial Medical Center AND Alzheimer’s disease
searched query  South Shore Hospital-South Weymouth AND Alzheimer’s disease
searched query  Cape Cod Hospital AND Alzheimer’s disease
searched query  Adcare Hospital of Worcester AND Alzheimer’s disease
searched query  Anna Jaques Hospital AND Alzheimer’s disease
searched query  Arbour-Fuller Hospital AND Alzheimer’s d

In [21]:
from IPython.display import display
for df in all_hospitals_tweet_data,all_hospitals_users_df,all_hospitals_tags_df,all_hospitals_query_sentiment_df:
    print(" ")
    print(len(df))
    display(df.head(2))

 
21


,tweet_id,twitter_handle,tweet_text,created_at,retweet_count,likes_count
0,1590087141446631430,DrNeilLove,On the next episode of our “Oncology Today” po...,2022-11-08 21:01:31+00:00,0,2
1,1589347138948956160,OncLiveSOSS,"Laura Spring, MD, of Massachusetts General Hos...",2022-11-06 20:01:01+00:00,0,0


 
21


,user_id,user_handle,user_name,user_profile_image,description,followers_count,created_at
0,334081266,DrNeilLove,Research To Practice,http://pbs.twimg.com/profile_images/1844038234...,RTP focuses on an integrated approach to oncol...,2705,2011-07-12 15:06:58+00:00
1,824013573873668096,OncLiveSOSS,OncLive SOSS,http://pbs.twimg.com/profile_images/8243550450...,Premier oncology summits that focus on the mos...,2383,2017-01-24 21:58:40+00:00


 
21


,tweet_id,tags
0,1590087141446631430,
1,1589347138948956160,


 
21


,tweet_id,query,sentiment
0,1590087141446631430,Massachusetts General Hospital AND Cancer,0.250
1,1589347138948956160,Massachusetts General Hospital AND Cancer,0.025


In [22]:
tweet_data = pd.concat([all_medicine_tweet_data,all_hospitals_tweet_data]).reset_index(drop = True)
tags_df = pd.concat([all_medicine_tags_df,all_hospitals_tags_df]).reset_index(drop = True)
users_df = pd.concat([all_medicine_users_df,all_hospitals_users_df]).reset_index(drop = True)


In [23]:
tags_df['tags'].value_counts()

                                                                                                        4004
neuropathy,diabetes,sciatica,chronicpain,pain,health,nervepain,chronicillness,neuropathydr,neurology      59
arthritis                                                                                                 31
PARADISE                                                                                                  27
OriginalResearch,AHA2022,CIRCAHA22,HOST                                                                   26
                                                                                                        ... 
cbd,cbdoil,cbdlifestyle,arthritis,pain,painrelief,otc,health                                               1
cbd,arthritis                                                                                              1
fibromyalgia,arthritis                                                                                     1
RA                 

In [24]:
tags_df['tags']=tags_df['tags'].replace({'':pd.NA})

In [25]:
all_medicine_query_sentiment_df['type'] = 'medicine'
all_hospitals_query_sentiment_df['type'] = 'hospital'

query_sentiment_df = pd.concat([all_medicine_query_sentiment_df,all_hospitals_query_sentiment_df]).reset_index(drop = True)

In [26]:
import sqlite3
conn = sqlite3.connect('Medicines Database DMDD2.db')

In [30]:
# Load data to the database

users_df.to_sql(name="Twitter_User", con=conn, if_exists='replace', index=False)
tweet_data.to_sql(name="Tweets", con=conn, if_exists='replace', index=False)
tags_df.to_sql(name="Tags", con=conn, if_exists='replace', index=False)
query_sentiment_df.to_sql(name="query_sentiment", con=conn, if_exists='replace', index=False)

4653